In [5]:
!pip install deepgram-sdk transformers soundfile

  Using cached deepgram_sdk-3.4.0-py3-none-any.whl (98 kB)
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Using cached websockets-12.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (130 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl (15 kB)
  Using cached aenum-3.1.15-py3-none-any.whl (137 kB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
  Using cached marshmallow-3.21.3-py3-none-any.whl (49 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)


In [7]:
import deepgram
import soundfile as sf
import transformers
from transformers import pipeline

In [8]:
DEEPGRAM_API_KEY = "********"
OPENAI_API_KEY = "************"


In [9]:
SYSTEM_PROMPT = "Here's what I found based on your query: "

In [17]:
class BackendVoiceConversation:
    def __init__(self, deepgram_config, tts_config, llm_config):
        from deepgram import DeepgramClient

        self.deepgram_dg = DeepgramClient(DEEPGRAM_API_KEY)

        if tts_config["name"] == "deepgram":
            self.tts_engine = self.deepgram_dg.tts
        elif tts_config["name"] == "openai":
            pass
        else:
            raise ValueError("Unsupported TTS engine:", tts_config["name"])

        if llm_config["name"] == "openai":
            self.llm_pipeline = pipeline("text-generation", model="text-davinci-003",
                                          api_key=OPENAI_API_KEY, temperature=0.7)
        else:
            raise ValueError("Unsupported LLM engine:", llm_config["name"])

    def process_audio_file(self, audio_filepath):
        audio_data, sample_rate = sf.read(audio_filepath)

        audio_bytes = audio_data.tobytes()

        text = self.deepgram_dg.transcription(audio_bytes, sample_rate=sample_rate)["channel"][0]["alternatives"][0]["transcript"]

        llm_response = self.llm_pipeline(SYSTEM_PROMPT + text, max_length=1024, truncation=True)[0]["generated_text"]

        tts_audio = self.tts_engine.synthesize_text(llm_response)

        stt_time = 1.0
        llm_time = 1.0
        total_time = stt_time + llm_time

        print(f"STT processing time: {stt_time:.2f} seconds (estimated)")
        print(f"LLM response time: {llm_time:.2f} seconds (estimated)")
        print(f"Total response time: {total_time:.2f} seconds (estimated)")

        print(f"LLM response: {llm_response}")


In [19]:
audio_file = r"C:\Users\bhara\Downloads\voice_command.mp3"

conversation = BackendVoiceConversation(
    deepgram_config={"name": "deepgram"},
    tts_config={"name": "deepgram"},
    llm_config={"name": "openai"}
)

conversation.process_audio_file(audio_file)


AttributeError: 'DeepgramClient' object has no attribute 'tts'